In [1]:
import time
import pickle
import os
import numpy as np
import pandas as pd
from IPython.display import clear_output
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
import options
import expectations
import iterators
import utils as ut

In [2]:
# numpy seed setup
seed = 29
np.random.seed(seed)

In [3]:
############# run parameters ###############
run_id = 'butterfly_bin_fine'
S0 = 1.
alpha = 1.  # risk aversion parameter
bf_level = 0.1 # higher payoff of the butterfly option
sigma = .20 # volatility of the process (absolute)
m = .05   # drift of the process (absolute)
T = .5 # maturity
step_nr = 200 # steps in the discretization
delta_x = 0.01 # step size of the spatial discretization
drop_optim = False # set to True to save the plots of the optimizers at each step

In [4]:
# expectation factory
def make_expect():
    return expectations.binomial_model(sigma=sigma)

# defining the risk measure
step_size = T / step_nr
rho_theta = expectations.RhoThetaCertain(alpha=alpha, m=m, t=step_size)

# defining the option
option = options.ButterflyOption(strike=S0, level=bf_level)

In [ ]:
# defining the spatial grid
error_propagation = np.absolute(m) * T + sigma * np.sqrt(T * step_nr)
print(f"Error propagation from the bounds of the space interval: {error_propagation:.3f}")

# initializing the space discretization
x_levels = S0 + np.arange(-.25 - error_propagation, .26 + error_propagation, delta_x)
print(f"Range of the space discretization: [{np.round(x_levels[0], 3)}, {np.round(x_levels[-1], 3)}]. Discretization step: {np.round(x_levels[1] - x_levels[0], 3)}. Number of points: {x_levels.shape[0]}")

# initializing the space discretization for the fine interpolation
x_fine = S0 + np.arange(-.25 - error_propagation, .25 + error_propagation, 0.001)
print(f"Range of the fine space discretization: [{np.round(x_fine[0], 3)}, {np.round(x_fine[-1], 3)}]. Discretization step: {np.round(x_fine[1] - x_fine[0], 3)}. Number of points: {x_fine.shape[0]}")

# plot coordinates
x_plot = S0 + np.arange(-.25, .26, 0.01)

In [6]:
# performing the iteration
iter_dict, theta_dict, expect_dict = iterators.certain_iterations(x_levels=x_levels, x_fine=x_fine, option=option, step_nr=step_nr, rho_theta=rho_theta, make_expect=make_expect)

In [7]:
# setting latex style for plots
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = 13
plt.rcParams['legend.fontsize'] = 13

Plotting the resulting approximated $S(T)f$.

In [ ]:
bachelier_levels = option.bachelier_price(x_plot, T, sigma)
model_levels = np.interp(x_plot, x_fine, iter_dict[step_nr])

plt.plot(x_plot, option.payoff(x_plot), color='blue', label=f'$f$')
plt.plot(x_plot, model_levels, color='green', label=r'$I(T/n)^nf$')
plt.plot(x_plot, bachelier_levels, color='red', linestyle='--', label=f'Bachelier price')
plt.xlabel('$x$')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Mean square error: {np.mean(np.power(model_levels - bachelier_levels, 2)):.4e}")

In [9]:
if drop_optim:
    # checking (or creating) the output folder
    plots_dir = os.path.join(os.path.pardir,'output', run_id, 'optimizers')
    ut.check_dir(plots_dir)

    # plotting the hedging strategy and the worst-case volatility at each step of the iteration
    for i in range(1, step_nr + 1):

        hedging = np.interp(x_plot, x_levels, theta_dict[i])
        plt.plot(x_plot, hedging, color='tab:blue')
        plt.xlabel('$S$')
        plt.ylabel(r'$\theta^*$')
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, f'iteration_{i:3d}.png'), bbox_inches='tight')
        plt.close()

In [10]:
# saving the trading strategy and the iterations for later use
dict_dir = os.path.join(os.path.pardir,'output', run_id)
out_dict = {
    "option": option,
    "maturity": T,
    "step_nr": step_nr,
    "training_model": make_expect().model,
    "delta_x": delta_x,
    "x_theta": x_levels,
    "theta_trained": theta_dict,
    "x_I": x_fine,
    "I_trained": iter_dict
}

# checking (or creating) the output folder
out_dir = os.path.join(os.path.pardir,'output', run_id)
ut.check_dir(out_dir)

# Save the dictionary to a pickle binary file
with open(os.path.join(dict_dir,'trained_model.pkl'), 'wb') as pickle_file:
    pickle.dump(out_dict, pickle_file)